In [56]:
# 1 导入包
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import sklearn
from sklearn.preprocessing import normalize
from keras.layers import LSTM
from keras.layers import Embedding, SimpleRNN


In [62]:
# 2 读取数据集

# dataframe = pd.read_csv("total.csv",header=None).values # 取消第一行作为表头
train_set = pd.read_csv("train_set.csv")
test_set = pd.read_csv("test_set.csv")
print(train_set)

# 3 读标签
# label = dataframe.loc[:,['label']]
y_train = train_set['label']
y_test = test_set['label']

print(y_train)



        label  frame.len  ip.hdr_len  ip.dsfield  ip.len  ip.flags.rb  \
0           0         66          20           0      52            0   
1           0         66          20           0      52            0   
2           0         66          20           0      52            0   
3           0         66          20           0      52            0   
4           0         66          20           0      52            0   
...       ...        ...         ...         ...     ...          ...   
837180     22         74          20           0      60            0   
837181     22         74          20           0      60            0   
837182     22         74          20          40      60            0   
837183     22         66          20           0      52            0   
837184     22         74          20           0      60            0   

        ip.flags.df  ip.flags.mf  ip.frag_offset  ip.ttl  ...  \
0                 0            0               0      63  

In [69]:

x_train_set_GA = train_set[['frame.len', 'ip.len', 'ip.flags.df', 'ip.flags.mf', 'ip.ttl',
       'ip.proto', 'tcp.hdr_len', 'tcp.flags.ns', 'tcp.flags.ecn',
       'tcp.flags.urg', 'tcp.flags.ack', 'tcp.flags.push', 'tcp.flags.reset',
       'tcp.flags.syn', 'tcp.flags.fin', 'tcp.window_size_value',
       'tcp.urgent_pointer', 'tcp.options.wscale.shift', 'tcp.options.mss_val',
       'tcp.options.timestamp.tsval', 'tcp.options.timestamp.tsecr',
       'udp.length', 'udp.checksum', 'tcp.option_kind',
       '_ws.col.Protocol_AJP13', '_ws.col.Protocol_BGP',
       '_ws.col.Protocol_BZR', '_ws.col.Protocol_DRDA', '_ws.col.Protocol_DSI',
       '_ws.col.Protocol_FTP', '_ws.col.Protocol_GIOP',
       '_ws.col.Protocol_GTPv2', '_ws.col.Protocol_Gopher',
       '_ws.col.Protocol_Gryphon', '_ws.col.Protocol_HTTP',
       '_ws.col.Protocol_HTTP/JSON', '_ws.col.Protocol_HTTP/XML',
       '_ws.col.Protocol_ICAP', '_ws.col.Protocol_ICMP',
       '_ws.col.Protocol_Kafka', '_ws.col.Protocol_LDAP',
       '_ws.col.Protocol_MySQL', '_ws.col.Protocol_NBNS',
       '_ws.col.Protocol_NDMP', '_ws.col.Protocol_NNTP',
       '_ws.col.Protocol_POP', '_ws.col.Protocol_Portmap',
       '_ws.col.Protocol_R3', '_ws.col.Protocol_RPC', '_ws.col.Protocol_RSYNC',
       '_ws.col.Protocol_RTMP', '_ws.col.Protocol_RTSP',
       '_ws.col.Protocol_SABP', '_ws.col.Protocol_SMTP',
       '_ws.col.Protocol_SSH', '_ws.col.Protocol_SSLv2',
       '_ws.col.Protocol_SSLv3', '_ws.col.Protocol_TCP',
       '_ws.col.Protocol_TCPCL', '_ws.col.Protocol_TDS',
       '_ws.col.Protocol_TELNET', '_ws.col.Protocol_TLSv1',
       '_ws.col.Protocol_TLSv1.1', '_ws.col.Protocol_TLSv1.2',
       '_ws.col.Protocol_TN3270', '_ws.col.Protocol_TPKT',
       '_ws.col.Protocol_UDP', '_ws.col.Protocol_VICP',
       '_ws.col.Protocol_ZEBRA']]
x_test_set = test_set[['frame.len', 'ip.len', 'ip.flags.df', 'ip.flags.mf', 'ip.ttl',
       'ip.proto', 'tcp.hdr_len', 'tcp.flags.ns', 'tcp.flags.ecn',
       'tcp.flags.urg', 'tcp.flags.ack', 'tcp.flags.push', 'tcp.flags.reset',
       'tcp.flags.syn', 'tcp.flags.fin', 'tcp.window_size_value',
       'tcp.urgent_pointer', 'tcp.options.wscale.shift', 'tcp.options.mss_val',
       'tcp.options.timestamp.tsval', 'tcp.options.timestamp.tsecr',
       'udp.length', 'udp.checksum', 'tcp.option_kind',
       '_ws.col.Protocol_AJP13', '_ws.col.Protocol_BGP',
       '_ws.col.Protocol_BZR', '_ws.col.Protocol_DRDA', '_ws.col.Protocol_DSI',
       '_ws.col.Protocol_FTP', '_ws.col.Protocol_GIOP',
       '_ws.col.Protocol_GTPv2', '_ws.col.Protocol_Gopher',
       '_ws.col.Protocol_Gryphon', '_ws.col.Protocol_HTTP',
       '_ws.col.Protocol_HTTP/JSON', '_ws.col.Protocol_HTTP/XML',
       '_ws.col.Protocol_ICAP', '_ws.col.Protocol_ICMP',
       '_ws.col.Protocol_Kafka', '_ws.col.Protocol_LDAP',
       '_ws.col.Protocol_MySQL', '_ws.col.Protocol_NBNS',
       '_ws.col.Protocol_NDMP', '_ws.col.Protocol_NNTP',
       '_ws.col.Protocol_POP', '_ws.col.Protocol_Portmap',
       '_ws.col.Protocol_R3', '_ws.col.Protocol_RPC', '_ws.col.Protocol_RSYNC',
       '_ws.col.Protocol_RTMP', '_ws.col.Protocol_RTSP',
       '_ws.col.Protocol_SABP', '_ws.col.Protocol_SMTP',
       '_ws.col.Protocol_SSH', '_ws.col.Protocol_SSLv2',
       '_ws.col.Protocol_SSLv3', '_ws.col.Protocol_TCP',
       '_ws.col.Protocol_TCPCL', '_ws.col.Protocol_TDS',
       '_ws.col.Protocol_TELNET', '_ws.col.Protocol_TLSv1',
       '_ws.col.Protocol_TLSv1.1', '_ws.col.Protocol_TLSv1.2',
       '_ws.col.Protocol_TN3270', '_ws.col.Protocol_TPKT',
       '_ws.col.Protocol_UDP', '_ws.col.Protocol_VICP',
       '_ws.col.Protocol_ZEBRA']]

print(x_train_set_GA)
print("筛选GA特征",x_train_set_GA.shape) # 筛选了69个特征

########## 一般是转为浮点数向量

# train_set = np.array(train_set[1:,:]).astype(np.float32) # 去掉第一行字段 
# test_set = np.array(test_set[1:,:]).astype(np.float32)


print(type(x_train_set_GA))
print(type(x_test_set))

# print(x_train_set_GA)


# 5 特征归一化  用MLP的时候用，LSTM不用，用embedding层 ,但是这里有两列数值太大了，
min_max_scaler = sklearn.preprocessing.MinMaxScaler(feature_range=(0, 20000))

x_train_set_GA[['tcp.options.timestamp.tsval','tcp.options.timestamp.tsecr','tcp.option_kind']] = min_max_scaler.fit_transform(x_train_set_GA[['tcp.options.timestamp.tsval','tcp.options.timestamp.tsecr','tcp.option_kind']])
x_test_set[['tcp.options.timestamp.tsval','tcp.options.timestamp.tsecr','tcp.option_kind']] = min_max_scaler.fit_transform(x_test_set[['tcp.options.timestamp.tsval','tcp.options.timestamp.tsecr','tcp.option_kind']])


train_set_GA = np.array(x_train_set_GA).astype(np.int64) # 转为整数向量
test_set_GA = np.array(x_test_set).astype(np.int64)



        frame.len  ip.len  ip.flags.df  ip.flags.mf  ip.ttl  ip.proto  \
0              66      52            0            0      63         6   
1              66      52            0            0      63         6   
2              66      52            0            0      63         6   
3              66      52            0            0      63         6   
4              66      52            0            0      63         6   
...           ...     ...          ...          ...     ...       ...   
837180         74      60            0            0      56         6   
837181         74      60            0            0      49         6   
837182         74      60            1            0      33         6   
837183         66      52            1            0     128         6   
837184         74      60            1            0      52         6   

        tcp.hdr_len  tcp.flags.ns  tcp.flags.ecn  tcp.flags.urg  ...  \
0              32.0           0.0            0.0   

C:\Users\jinghan\AppData\Local\Temp\ipykernel_12636\1553723407.py:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_train_set_GA[['tcp.options.timestamp.tsval','tcp.options.timestamp.tsecr']] = min_max_scaler.fit_transform(x_train_set_GA[['tcp.options.timestamp.tsval','tcp.options.timestamp.tsecr']])
C:\Users\jinghan\AppData\Local\Temp\ipykernel_12636\1553723407.py:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_test_set[['tcp.options.timestamp.tsval','tcp.options.timestamp.tsecr']] = min_max_scaler.f

In [72]:
print(train_set_GA[:,20])

[ 9251  8951  4468 ... 20000     0     0]


In [73]:

# 6 标签转化为one-hot编码 
y_train = keras.utils.to_categorical(y_train, num_classes=23)
y_test = keras.utils.to_categorical(y_test, num_classes=23)

print("y_train.shape",y_train.shape)


y_train.shape (837185, 23)


In [75]:

########## Embedding 层输入整数向量

# train_set = np.array(train_set).astype(np.int32) # 转成整数
# x_test_set = np.array(x_test_set).astype(np.int32)


print(train_set_GA)
print(type(test_set_GA))

[[66 52  0 ...  0  0  0]
 [66 52  0 ...  0  0  0]
 [66 52  0 ...  0  0  0]
 ...
 [74 60  1 ...  0  0  0]
 [66 52  1 ...  0  0  0]
 [74 60  1 ...  0  0  0]]
<class 'numpy.ndarray'>


In [77]:
##################  LSTM
max_features = 30000 # 作为特征的单词索引个数
# x_train = x_train[:, None]

print(train_set.shape)

model = Sequential()
# model.add(Dense(64, activation='relu', input_dim=104))
# model.add(Dropout(0.5))
model.add(Embedding(max_features, 32)) # 接收二维整数向量，输出三维向量
# model.add(Dropout(0.5))
# model.add(Dense(32, activation='relu'))
model.add(LSTM(32)) # 接收三维向量
model.add(Dense(23, activation='sigmoid')) 


# loss = [binary_crossentropy,categorical_crossentropy]
# optimizer = [rmsprop,] RMSProp对比Adamgrad增加了指数平滑
model.compile(optimizer='rmsprop',loss='categorical_crossentropy',metrics=['acc'])

model.fit(train_set_GA, y_train ,epochs=5, batch_size=128, validation_split=0.2)
score = model.evaluate(test_set_GA, y_test, batch_size=128)

#################


(837185, 97)
Epoch 1/5


InvalidArgumentError: Graph execution error:

Detected at node 'sequential_10/embedding_10/embedding_lookup' defined at (most recent call last):
    File "c:\ProgramData\Anaconda3\envs\tf\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "c:\ProgramData\Anaconda3\envs\tf\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "c:\ProgramData\Anaconda3\envs\tf\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
      app.launch_new_instance()
    File "c:\ProgramData\Anaconda3\envs\tf\lib\site-packages\traitlets\config\application.py", line 976, in launch_instance
      app.start()
    File "c:\ProgramData\Anaconda3\envs\tf\lib\site-packages\ipykernel\kernelapp.py", line 677, in start
      self.io_loop.start()
    File "c:\ProgramData\Anaconda3\envs\tf\lib\site-packages\tornado\platform\asyncio.py", line 199, in start
      self.asyncio_loop.run_forever()
    File "c:\ProgramData\Anaconda3\envs\tf\li